<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0b937712d4f4a3cd61b426c6d6bfe78b06939814fc2aa5618327df1c3a4fa967
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-20 13:00:33
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  1.16 L
Current:  1.45 C
-------------------
Today PnL: 30.97 K (0.21%)
Current PnL: -21.06 L (-13.8%)
CY Booked + Current PnL: -7.10 L (-4.65%)
-------------------
Total profit:  1.85 L
Total loss:  -22.91 L
-------------------
Total Booked + Current PnL: 19.88 L (15.82%)
Total Booked PnL: 40.94 L (32.59%)
Curr Year Booked PnL: 13.96 L (9.65%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 85.97 L (59.4%)
Deployed:  1.26 C
Current:  1.45 C
CAGR/XIRR %: 8.14%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.22,-6.99,7.53,0.02,12226.0,-12200.0,162359.0,147.77,78.0,M-SC,3.08,234.0,-1.00,1.13,41.22,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.14,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.03,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,0.94,10.70,8.13,19.69,17256.0,20508.0,212250.0,-0.11,58.0,X-MC,3.65,78.0,1.19,1.48,18.22,XY25,NTT,AC
35,ICICIGI,2252.93,0.36,9.32,9.95,20.19,18961.0,16246.0,190566.0,-14.37,59.0,X-MC,7.72,68.0,0.86,1.33,25.84,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,0.08,12.84,7.58,21.40,22704.0,34094.0,299520.0,4.67,62.0,X-LC,8.06,12.0,1.50,2.09,20.34,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,2275.0,3.68,4.91,47.62,54.87,156315.0,15366.0,328254.0,-18.16,60.0,X-LC,3.62,2.0,0.10,2.29,13.60,X40,BTT,IT
0,5PAISA,593.0,2.80,-34.70,72.89,12.90,104005.0,-75808.0,142688.0,110.34,63.0,H-SC,15.67,161.0,-0.73,0.99,18.64,OX40N,NTT,FINANCE
15,CAMS,4762.0,2.18,5.20,18.62,24.79,49335.0,13101.0,264957.0,0.31,61.0,X-SC,2.25,86.0,0.27,1.85,29.54,X40N,NTT,MISC
37,IEX,219.0,2.09,-5.00,56.45,48.63,107228.0,-10001.0,189953.0,-36.71,51.0,H-SC,15.33,136.0,-0.09,1.32,5.97,XR,NTT,MISC
57,RAJESHEXPO,518.0,2.04,-62.77,168.81,0.07,86205.0,-86111.0,51066.0,1771.21,65.0,L-SC,2.21,268.0,-1.00,0.36,27.00,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.0,-3.81,-27.74,97.67,42.84,45881.0,-18030.0,46976.0,-52.62,39.0,X-SC,37.19,83.0,-0.39,0.33,1.78,XY24,NTT,MISC
81,VALIANTORG,838.0,-3.09,-43.91,210.49,74.15,193158.0,-71839.0,91766.0,-265.32,29.0,H-SC,13.34,149.0,-0.37,0.64,17.63,XR,NTT,CHEMICALS
80,VAIBHAVGBL,521.0,-2.35,-24.24,112.22,60.77,155382.0,-44313.0,138462.0,59.78,45.0,H-SC,11.03,159.0,-0.29,0.96,24.70,XR,NTT,JEWELLERY
47,KANSAINER,340.0,-2.33,-20.18,42.16,13.47,90747.0,-54423.0,215244.0,-67.15,34.0,H-SC,1.73,158.0,-0.60,1.50,9.45,XY24,NTT,PAINTS
6,ASIANTILES,137.0,-2.01,-12.70,108.11,81.67,88535.0,-11917.0,81893.0,7361.11,52.0,L-SC,18.05,271.0,-0.13,0.57,58.59,XR,NTT,CERAMICS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.43,-2.17,112.53,107.91,162537.0,-3205.0,144439.0,-20.55,41.0,M-SC,10.12,216.0,-0.02,1.01,0.76,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,1.22,-6.99,7.53,0.02,12226.0,-12200.0,162359.0,147.77,78.0,M-SC,3.08,234.0,-1.00,1.13,41.22,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,-0.14,-14.21,16.67,0.09,14413.0,-14317.0,86460.0,100.64,49.0,M-SC,4.03,253.0,-0.99,0.60,13.59,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-2.33,-20.18,42.16,13.47,90747.0,-54423.0,215244.0,-67.15,34.0,H-SC,1.73,158.0,-0.60,1.50,9.45,XY24,NTT,PAINTS
73,TATAELXSI,9161.00,0.35,-22.67,70.72,32.03,72102.0,-29881.0,101954.0,-16.37,50.0,H-SC,6.39,148.0,-0.41,0.71,14.20,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,0.07,1.15,70.56,72.51,107727.0,1730.0,152675.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.24,XR,NTT,IT
38,INDIAMART,4810.62,-0.28,3.41,99.90,106.72,127416.0,4208.0,127544.0,-48.72,44.0,H-SC,4.60,139.0,0.03,0.89,27.59,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.28,3.41,99.90,106.72,127416.0,4208.0,127544.0,-48.72,44.0,H-SC,4.60,139.0,0.03,0.89,27.59,AR,ATH,MISC
85,WIPRO,420.00,0.07,1.15,70.56,72.51,107727.0,1730.0,152675.0,-13.34,56.0,M-LC,5.08,99.0,0.02,1.06,7.24,XR,NTT,IT
25,FINCABLES,1641.55,-0.43,-2.17,112.53,107.91,162537.0,-3205.0,144439.0,-20.55,41.0,M-SC,10.12,216.0,-0.02,1.01,0.76,OX40N,ATH,CABLES
86,ZYDUSLIFE,1286.17,0.39,-1.52,37.73,35.63,77868.0,-3193.0,206381.0,-15.41,36.0,H-MC,5.02,119.0,-0.04,1.44,14.89,AR,ATH,PHARMA
84,WHIRLPOOL,2270.00,-1.17,-4.79,91.08,81.92,99547.0,-5502.0,109296.0,-44.29,32.0,M-SC,6.95,236.0,-0.06,0.76,29.39,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.18,-14.64,66.57,42.18,157059.0,-40479.0,235931.0,-23.08,20.0,X-LC,0.20,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
11,BATAINDIA,2096.00,-0.17,-34.84,107.48,35.20,90119.0,-44823.0,83847.0,9.89,24.0,X-SC,11.47,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.00,-18.41,67.72,36.84,70690.0,-23553.0,104385.0,-26.58,28.0,X-SC,1.45,90.0,-0.33,0.73,6.86,X40N,ATH,ELECTRICAL
46,JSWINFRA,342.00,0.42,-6.57,23.33,15.23,43669.0,-13169.0,187178.0,-20.15,30.0,X-MC,5.97,66.0,-0.30,1.30,23.56,X40N,NTT,REALTY
60,RELAXO,1176.00,-0.31,-46.60,185.23,52.31,143575.0,-67648.0,77512.0,-43.18,31.0,X-SC,3.29,91.0,-0.47,0.54,3.75,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,TMPV,600.00,-0.18,-14.64,66.57,42.18,157059.0,-40479.0,235931.0,-23.08,20.0,X-LC,0.20,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
43,ITC,452.00,0.43,-0.97,11.52,10.44,22832.0,-1946.0,198192.0,-40.74,45.0,X-LC,0.94,5.0,-0.09,1.38,4.54,X40,NTT,FMCG
36,ICICIPRULI,790.00,0.51,2.30,28.03,30.97,50677.0,4058.0,180796.0,-21.72,54.0,X-MC,1.44,75.0,0.08,1.26,15.14,X40,ATH,INSURANCE
34,HONAUT,58357.33,0.00,-18.41,67.72,36.84,70690.0,-23553.0,104385.0,-26.58,28.0,X-SC,1.45,90.0,-0.33,0.73,6.86,X40N,ATH,ELECTRICAL
66,SIEMENS,4671.50,0.42,-13.35,44.85,25.51,72323.0,-24840.0,161255.0,-0.97,60.0,H-LC,1.82,49.0,-0.34,1.12,18.83,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.17,-34.84,107.48,35.20,90119.0,-44823.0,83847.0,9.89,24.0,X-SC,11.47,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.07,-0.15,-5.34,33.17,26.07,51414.0,-8740.0,155000.0,-30.31,32.0,X-MC,8.44,55.0,-0.17,1.08,0.00,X40,ATH,APPARELS
76,TMPV,600.00,-0.18,-14.64,66.57,42.18,157059.0,-40479.0,235931.0,-23.08,20.0,X-LC,0.20,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.62,-12.71,70.07,48.45,122637.0,-25485.0,175021.0,-19.01,36.0,X-MC,7.08,64.0,-0.21,1.22,0.93,X40N,ATH,FINANCE
56,QUESS,424.00,-3.81,-27.74,97.67,42.84,45881.0,-18030.0,46976.0,-52.62,39.0,X-SC,37.19,83.0,-0.39,0.33,1.78,XY24,NTT,MISC


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-3.81,-27.74,97.67,42.84,45881.0,-18030.0,46976.0,-52.62,39.0,X-SC,37.19,83.0,-0.39,0.33,1.78,XY24,NTT,MISC
60,RELAXO,1176.00,-0.31,-46.60,185.23,52.31,143575.0,-67648.0,77512.0,-43.18,31.0,X-SC,3.29,91.0,-0.47,0.54,3.75,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.17,-34.84,107.48,35.20,90119.0,-44823.0,83847.0,9.89,24.0,X-SC,11.47,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
34,HONAUT,58357.33,0.00,-18.41,67.72,36.84,70690.0,-23553.0,104385.0,-26.58,28.0,X-SC,1.45,90.0,-0.33,0.73,6.86,X40N,ATH,ELECTRICAL
52,MEDANTA,1486.00,0.40,0.51,23.44,24.07,25960.0,563.0,110750.0,-7.57,39.0,X-SC,4.67,89.0,0.02,0.77,20.23,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.07,-14.41,39.36,19.28,116545.0,-49856.0,296100.0,-25.24,63.0,X-LC,7.94,1.0,-0.43,2.06,9.46,X40,ATH,IT
41,INFY,2275.00,3.68,4.91,47.62,54.87,156315.0,15366.0,328254.0,-18.16,60.0,X-LC,3.62,2.0,0.10,2.29,13.60,X40,BTT,IT
76,TMPV,600.00,-0.18,-14.64,66.57,42.18,157059.0,-40479.0,235931.0,-23.08,20.0,X-LC,0.20,3.0,-0.26,1.64,0.00,XY24,NTT,AUTO
82,VBL,671.64,-0.48,-8.63,48.48,35.67,139913.0,-27243.0,288599.0,-19.52,43.0,X-LC,6.01,4.0,-0.19,2.01,4.03,X40N,ATH,FMCG
43,ITC,452.00,0.43,-0.97,11.52,10.44,22832.0,-1946.0,198192.0,-40.74,45.0,X-LC,0.94,5.0,-0.09,1.38,4.54,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,COFFEEDAY,80.00,0.79,-37.84,117.63,35.27,83022.0,-42970.0,70579.0,-51.71,34.0,L-SC,19.44,270.0,-0.52,0.49,72.74,XR,NTT,HOTELS
6,ASIANTILES,137.00,-2.01,-12.70,108.11,81.67,88535.0,-11917.0,81893.0,7361.11,52.0,L-SC,18.05,271.0,-0.13,0.57,58.59,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.49,-11.87,58.33,39.54,52004.0,-12005.0,89154.0,-31.03,71.0,M-SC,6.85,220.0,-0.23,0.62,25.72,AR,ATH,AUTO
51,MASFIN,398.61,-0.83,-4.64,27.99,22.05,26152.0,-4545.0,93435.0,-17.80,53.0,H-SC,7.16,164.0,-0.17,0.65,35.81,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.30,-21.90,57.53,23.03,80463.0,-39209.0,139862.0,51.32,43.0,H-SC,12.20,167.0,-0.49,0.97,38.77,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,1.22,-6.99,7.53,0.02,12226.0,-12200.0,162359.0,147.77,78.0,M-SC,3.08,234.0,-1.00,1.13,41.22,OX40N,NTT,PAINTS
13,BSOFT,831.70,-1.10,-29.84,114.91,50.79,113399.0,-41968.0,98685.0,-9.93,58.0,H-SC,7.91,151.0,-0.37,0.69,14.90,XR,ATH,IT
5,ASIANPAINT,3460.25,-1.16,-0.03,20.99,20.95,52828.0,-88.0,251680.0,-1.05,75.0,X-LC,11.46,31.0,-0.00,1.75,35.59,X40,ATH,PAINTS
32,HCLTECH,1922.01,-0.59,7.26,16.29,24.73,42271.0,17566.0,259490.0,11.91,73.0,X-LC,4.49,7.0,0.42,1.81,23.60,X40,ATH,IT
68,SONACOMS,806.63,0.49,-11.87,58.33,39.54,52004.0,-12005.0,89154.0,-31.03,71.0,M-SC,6.85,220.0,-0.23,0.62,25.72,AR,ATH,AUTO


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.93
1,25,44.97
2,50,75.86


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.74
MC    29.20
LC    25.07
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.09
X40      21.00
X40N     11.64
XR        9.48
XY25      9.16
AR        9.14
OX40N     7.76
X200      1.91
SR        1.01
MH        0.82
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    24.33
X-MC    22.72
X-LC    20.00
M-SC    12.14
X-SC     7.85
H-MC     4.76
H-LC     3.03
L-SC     1.42
M-MC     1.39
M-LC     1.06
L-LC     0.98
L-MC     0.33
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.84,-4.18,37.87
IT,13.71,-13.25,71.76
FINANCE,9.52,-13.45,61.39
MISC,7.50,-20.69,70.14
PAINTS,6.04,-6.45,22.26
ELECTRICAL,5.69,-10.58,49.79
INSURANCE,4.51,2.38,32.03
PHARMA,3.66,-0.80,33.78
AUTO,3.35,-19.36,70.54


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2983007.0,21
XR,1266521.0,13
AR,1260522.0,10
X40,989141.0,14
X40N,701089.0,9
OX40N,677523.0,10
XY25,327258.0,6
SR,275642.0,2
MH,66444.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3485250.0,25
M-SC,1314052.0,15
X-MC,1168211.0,16
X-LC,892483.0,11
X-SC,714069.0,8
H-MC,395141.0,3
L-SC,257762.0,3
H-LC,122709.0,2
M-LC,107727.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1187285.0      6
           AR         855894.0      5
           XR         784559.0      7
M-SC       XY24       769899.0      6
X-MC       X40        424829.0      7
X-LC       X40        390791.0      5
X-MC       XY24       340525.0      3
H-SC       OX40N      315426.0      4
X-SC       XY24       276948.0      3
M-SC       OX40N      275892.0      5
H-SC       SR         275642.0      2
X-SC       X40N       263600.0      3
X-MC       X40N       231256.0      4
X-LC       XY24       224665.0      2
H-MC       AR         211456.0      2
X-LC       X40N       206233.0      2
H-MC       XY24       183685.0      1
X-SC       X40        173521.0      2
X-MC       XY25       171601.0      2
L-SC       XR         171557.0      2
M-SC       XR         147412.0      2
           AR         120849.0      2
M-LC       XR         107727.0      1
L-SC       OX40N       86205.0      1
H-LC       AR          72323.0      1
X-LC       XY25        70794.0      2
H-SC       MH          66444.0      1
L-MC       XR          55266.0      1
H-LC       X200        50386.0      1
M-MC       XY25        46017.0      1
L-LC       XY25        38846.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
